In [18]:
import numpy as np
import dimod
from itertools import product
from collections.abc import Iterable
from pyqubo import Binary
import neal


class Prescription():
    def __init__(self, water, nutrients):
        
        if isinstance(water,Iterable):
            assert (len(water)==len(nutrients))
        
        self.water = water
        self.nutrients = nutrients
    
    def __len__(self):
        return len(self.water)
    
    def get_yield(self, w_range=12, n_range=12, w_peak=6,n_peak=8):
        a,b,c,d,e,f = 0.413,1.949,1.352,-2.472,1.218,-2.033
        # function maximum if concave (down)
        maxn = (2*f*b/e - c)/(e - 4*f*d/e)
        maxw = (2*d*c - e*b) / (e**2 - 4*d*f)
        # normalize water and nutrients
        w_min = w_peak - w_range*maxw
        n_min = n_peak - n_range*maxn
    
        w = (self.water - w_min) / w_range
        n = (self.nutrients - n_min) / n_range
    
        return (a + b*n + c*w + d*(n)**2 + e*n*w + f*(w)**2)

    

In [19]:
# available water and nutrients
Wtotal = 24
Ntotal = 32

ngridcells = 4

gridcells = np.arange(ngridcells)

# possible values of water, as integers
water = np.array([4,5,6,7])
w_peak=6
w_range=12 # sets approx scaling of integer labels corresponding to 0 to 1

# possible values of nutrients, as integers
nutrients = np.array([5,6,7,8,9,10])
n_peak=8
n_range=12 # "0" to 1

# these values map to these yields, with optimal at w=2, n=2
#field_yield = get_yield(*np.array([i for i in product(water,nutrients)]).T,
#                        w_range=w_range,
#                        w_peak=w_peak,
#                        n_peak=n_peak,
#                        n_range=n_range).reshape(-1,len(nutrients))

prescription = Prescription(*np.array(list(product(water, nutrients))).T)
# always try highest yield first
bias = np.sort(prescription.get_yield())[::-1]

In [20]:
# different yields for different grid cells
eta = [1.0*100,
       0.7*100,
       0.5*100,
       0.9*100]
#eta = [100.,100.,100.,100.]
#eta = [100.,1.]
#eta = [100]*4/


# number of grid cells in field
ngridcells = len(eta)

# ZONE definitions for fixed irrigation
#waterzones = {'a':[0,1,2,3,4], 'b':[5,6,7,8,9,10,11],
#              'c':[12,13,14,15,16,17,18,19,20,21,22,23,24],'d':[25,26,27,28,29,30]}

waterzones = {'a':[0,2], 'b':[1,3]}

try:
    assert (np.sort(np.array([v1 for v in waterzones.values() for v1 in v ]).flatten()) == np.arange(ngridcells)).all()
except:
    print('ERROR. Check that all grid cells have been assigned to a water zone.')


# CONSTRAINTS    
ll1, ll2 = 300, 20
onehotconstraint, zonelinking = True, True
ll3, ll4 = 100,100
waterconstraint, nutrientconstraint = True, True






In [21]:
H = 0
Wused=0
Nused=0
field_yield = prescription.get_yield()

f = 0

for g in range(ngridcells):
    
    # only one prescription (nutrient+water) allowed per grid cell
    index = np.array([Binary('{}_{}_{}'.format(g,w,n)) 
                      for w,n in zip(prescription.water,prescription.nutrients)])
    H -= (eta[g]*prescription.get_yield()*index).sum()
    
    # one-hot restriction
    onehot = index.sum()
    # water used
    Wused+=(prescription.water*index).sum()
    # nutrients used
    Nused+=(prescription.nutrients*index).sum()
    if onehotconstraint:
        H += ll1*(onehot - 1)**2
    
#zone-linking constraint
# penalize if water j != water j' when looking at cells in the same zone

for zone, gridcells in waterzones.items():
    # there are no linking constraints needed
    if len(gridcells)==1:
        continue
    # first grid cell. Comparing all others to this first grid cell.
    g = gridcells[0]
    for ww in water:
        # these are allowed
        allowed = np.array([Binary('{}_{}_{}'.format(g,ww,n))*Binary('{}_{}_{}'.format(g1,ww,n1))
                   for g1 in gridcells[1:] for n in prescription.nutrients 
                   for n1 in prescription.nutrients])
        H -= ll2*allowed.sum()

"""
for w in range(len(water)):
    # for water value w, look at all water zones
    for key,value in waterzones.items():
        # if there is only one grid cell in the zone, skip to the next zone
        if len(value)==1:
            continue
        # use the first grid cell within the zone for the constraint
        sum_0 = 0
        for n in range(len(nutrients)):
            index = '{}_{}_{}'.format(value[0],w,n)
            sum_0 += Binary(index)
        for f in value[1:]:
            sum_i = 0
            for n in range(len(nutrients)):
                index = '{}_{}_{}'.format(f,w,n)
                sum_i += Binary(index)
                #print(index)
            if zonelinking:
                H += lambda2*(sum_0 - sum_i)**2
"""            
# water constraint (possible values: 0 to 15)
slackVar = 8*Binary('I8w')+4*Binary('I4w')+2*Binary('I2w') + 1*Binary('I1w')
if waterconstraint:
    H+=ll3*(Wused + slackVar - Wtotal)**2

# nutrient constraint (possible values: 0 to 15)
slackVar = 8*Binary('I8n')+4*Binary('I4n')+2*Binary('I2n') + 1*Binary('I1n')
if nutrientconstraint:
    H+=ll4*(Nused + slackVar - Ntotal)**2


In [22]:
model = H.compile()
#bqm = model.to_bqm()
bqm = model.to_dimod_bqm()

sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=1000)

In [23]:
best_sample = sampleset.first


In [24]:
residual = {'w':0, 'n':0}
total_yield = 0
waterused = 0
nutrientsused = 0
print('Peak yield is at w={},n={}'.format(w_peak,n_peak))
for key,value in best_sample.sample.items():
    #print(key)
    if value==1:
        if (key[0]=='I'):
            kind = key[-1]
            residual[kind] = residual[kind] + int(key[1:-1])
        else:
            f,w,n = [int(i) for i in key.split('_')]
            total_yield += eta[f]*Prescription(w,n).get_yield()
            print('Grid cell {:} used {:} Water and {:} Nutrients (Yield {:.3f})'.format(
                f,w,n, eta[f]*Prescription(w,n).get_yield()))
            waterused +=w
            nutrientsused +=n
print('Water used: {:3d} (out of available {})'.format(waterused,Wtotal))
print('Nutri used: {:3d} (out of available {})'.format(nutrientsused,Ntotal))
print('\nUnused resources (residuals from inequality):')
print(residual)
print('\nWater zone restrictions:')
print(waterzones)
print('\nTotal Yield {:.3f}'.format(total_yield))

Peak yield is at w=6,n=8
Grid cell 0 used 4 Water and 6 Nutrients (Yield 115.154)
Grid cell 0 used 4 Water and 8 Nutrients (Yield 118.637)
Grid cell 1 used 6 Water and 5 Nutrients (Yield 76.184)
Grid cell 2 used 4 Water and 5 Nutrients (Yield 54.131)
Grid cell 3 used 6 Water and 7 Nutrients (Yield 110.311)
Water used:  24 (out of available 24)
Nutri used:  31 (out of available 32)

Unused resources (residuals from inequality):
{'w': 0, 'n': 1}

Water zone restrictions:
{'a': [0, 2], 'b': [1, 3]}

Total Yield 474.416
